In [0]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm

In [0]:
def share_path(S_0, r, sigma, Z, dT):
    return S_0 * np.exp(np.cumsum((r - sigma**2/2) * dT + sigma * np.sqrt(dT) * Z))

def european_call_payoff(S_T, K, r, T):
    """S_T is the price of the underlying at expiry.
    K is the strike price
    r is the risk free rate
    T is the time period till maturity"""
    return np.exp(-T*r) * np.maximum(0, S_T - K) #payoff for call option

def euro_uao_call(barrier, paths, K, r, T):
    """barrier is the barrier level,
    paths is a list of share price paths,
    K is the strike price
    r is the risk free rate
    T is the time period of each share price path"""
    prices = []
    for path in paths:
            if np.max(path) > barrier: # knocked out
                    prices.append(0)
            else:
                    prices.append(european_call_payoff(path, K, r, T)) 
    return np.mean(prices)
    
def sumUpPricePath(path, path_total):
    return [x + y for x, y in zip(path_total, path)]

In [0]:
T = 1
L = 150
S_0 = 100
K = 100
V_0 = 100
r = 0.08
sigma_s = 0.30
sigma_v = 0.25
debt = 175
corr = 0.2
recovery_rate = 0.25
frequency = 12
correlation = 0.2

corr_matrix = np.array([[1, correlation], [correlation, 1]])

In [4]:
# 1. Simulate paths for the underlying share and for the counterparty’s firm value using sample sizes of 1000, 2000, ..., 50000. 
# Do monthly simulations for the lifetime of the option.

for sampleSize in range(1000, 51000, 1000):
    share_path_total = [0] * frequency
    firm_value_total = [0] * frequency
    
    #for each sample size, sum up all price path for each simulation so that the mean can be calculated later
    for i in range(0, sampleSize):
        norm_matrix = norm.rvs(size=np.array([2, frequency]))
        corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)
        share_price_path = share_path(S_0, r, sigma_s, corr_norm_matrix[0,], T/frequency)
        firm_value_path = share_path(V_0, r, sigma_v, corr_norm_matrix[1,], T/frequency)
        share_path_total = sumUpPricePath(share_price_path, share_path_total)
        firm_value_total = sumUpPricePath(firm_value_path, firm_value_total)
        
    #get the mean path for the sum of all the simulations
    share_path_mean = list(map(lambda totalShare: totalShare/sampleSize, share_path_total))
    firm_value_mean = list(map(lambda totalShare: totalShare/sampleSize, firm_value_total))
    print("Sample Size: " + str(sampleSize))
    print("Share price path is " + str(share_path_mean))
    print("Firm value path is " + str(firm_value_mean))
    print("\n")

Sample Size: 1000
Share price path is [100.2376753842869, 100.89989946428187, 101.83751908182866, 102.14070700572785, 103.03955774581581, 103.46090259321656, 103.97433333949786, 104.41549547285598, 104.57223033186567, 105.78840672177382, 106.48719739525932, 107.14396523183045]
Firm value path is [100.57473639979655, 101.37382212414124, 102.03663156113754, 102.92285936668979, 103.47732672472358, 104.8075932410258, 105.76234528209456, 106.23510374174225, 107.0734346847371, 108.14010030498193, 108.99373048580203, 109.1198120502427]


Sample Size: 2000
Share price path is [101.09894479889321, 101.79540870165718, 102.09280225135635, 102.94666500801652, 103.62617149559073, 104.5785033548063, 105.2038833785617, 105.71030757385893, 106.18556280856333, 106.9033497324659, 107.57188660898613, 108.40406602532335]
Firm value path is [100.91259762007856, 101.3800618312032, 101.97028995785952, 103.00598316265163, 103.66984515695617, 104.35443655329915, 104.91515037604667, 105.69318461665164, 106.5579

In [0]:
#2. Determine Monte Carlo estimates of both the default-free value of the option and the Credit Valuation Adjustment (CVA).
#3. Calculate the Monte Carlo estimates for the price of the option incorporating counterparty risk, given by the default-free price less the CVA.

call_opt_val, cva_estimates, cva_std = [], [], []

for sampleSize in range(1000, 51000, 1000):
    share_path_total = [0] * frequency
    firm_value_total = [0] * frequency
    
    #for each sample size, sum up all price path for each simulation so that the mean can be calculated later
    for i in range(0, sampleSize):
        norm_matrix = norm.rvs(size=np.array([2, frequency]))
        corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)
        share_price_path = share_path(S_0, r, sigma_s, corr_norm_matrix[0,], T/frequency)
        firm_value_path = share_path(V_0, r, sigma_v, corr_norm_matrix[1,], T/frequency)
        share_path_total = sumUpPricePath(share_price_path, share_path_total)
        firm_value_total = sumUpPricePath(firm_value_path, firm_value_total)

    #get the mean path for the sum of all the simulations
    share_path_mean = list(map(lambda totalShare: totalShare/sampleSize, share_path_total))
    firm_value_mean = list(map(lambda totalShare: totalShare/sampleSize, firm_value_total))

    ################################################
    #######     terminal value of option   #########
    ################################################

    call_val = euro_uao_call(L, share_path_mean, K, r, T/frequency)
    call_opt_val.append(call_val)

    ################################################
    #######               CVA              #########
    ################################################

    amount_lost = [np.exp(-T/frequency*r) * (1-recovery_rate)*(term_firm_val < debt) * call_val for term_firm_val in firm_value_mean]
    cva_estimates.append(np.mean(amount_lost))
    cva_std.append(np.std(amount_lost) / np.sqrt(frequency))

In [6]:
#2a. Determine Monte Carlo estimates of the default-free value of the option 
#2b. Determine Monte Carlo estimates of the Credit Valuation Adjustment (CVA)
#3. Calculate the Monte Carlo estimates for counterparty risk

# print out the call_opt_val, cva_estimates and cva_std
call_opt_val = np.mean(call_opt_val)
cva_estimates = np.mean(cva_estimates)
cva_std = np.mean(cva_std)

counter_party_risk = call_opt_val - cva_estimates

print("The European up-and-out call option price is {}. \n".format(call_opt_val))
print("The Credit Valuation Adjustment  is {}. \n".format(cva_estimates))
print("The counter party risk is {}. \n".format(counter_party_risk))

The European up-and-out call option price is 4.4406869806665155. 

The Credit Valuation Adjustment  is 3.3083856478500953. 

The counter party risk is 1.1323013328164202. 

